In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from ast import literal_eval
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from collections import defaultdict

Read each fitting results file and print the fitting criteria and mean parameters

In [2]:
DATA_DIR = "../../data/fitting_results"
FIGURE_DIR = "../../figures"

In [3]:
# create a dictionary to store dataframes
agent_types = ['lqr', 'sparse_lqr', 'sparse_max_discrete', 'sparse_max_continuous', 'null_model_1', 'null_model_2', 'hill_climbing']
all_model_dfs = {}
for agent_type in agent_types:
    # read the aggregated fitting results
    df = pd.read_csv(f"{DATA_DIR}/fitting_results_{agent_type}.csv")
    all_model_dfs[agent_type] = df
    # print info about 
    print(f"agent type: {agent_type}, n={len(df)}")
    print(f"{agent_type} llh: {df['ll'].sum()}")
    print(f"{agent_type} AIC: {df['AIC'].sum()}")
    print(f"{agent_type} llh mean: {df['ll'].mean()}")
    print(f"{agent_type} AIC mean: {df['AIC'].mean()}")
    if agent_type == "lqr" or agent_type == "null_model_1":
        print(f"mean exp param: {df['exp_param'].mean()}")
        print(f"mean vm param: {df['vm_param'].mean()}")
    if agent_type == "null_model_1":
        print(f"mean n: {df['n'].mean()}")
        print(f"median n: {df['n'].median()}")
        print(f"mean b: {df['b'].mean()}")

agent type: lqr, n=111
lqr llh: -13064.787564000002
lqr AIC: 26573.575128000004
lqr llh mean: -117.70078886486486
lqr AIC mean: 239.40157772972967
mean exp param: 0.009439116295392144
mean vm param: 1.7997701788396911
agent type: sparse_lqr, n=111
sparse_lqr llh: -9031.817217
sparse_lqr AIC: 18729.634434
sparse_lqr llh mean: -81.36772267567568
sparse_lqr AIC mean: 168.7354453513514
agent type: sparse_max_discrete, n=111
sparse_max_discrete llh: -8190.639816
sparse_max_discrete AIC: 17269.279632
sparse_max_discrete llh mean: -73.78954789189193
sparse_max_discrete AIC mean: 155.57909578378386
agent type: sparse_max_continuous, n=111
sparse_max_continuous llh: -8364.688535000001
sparse_max_continuous AIC: 17617.377069999995
sparse_max_continuous llh mean: -75.35755436936938
sparse_max_continuous AIC mean: 158.7151087387387
agent type: null_model_1, n=111
null_model_1 llh: -8834.4462921
null_model_1 AIC: 18556.8925842
null_model_1 llh mean: -79.58960623513514
null_model_1 AIC mean: 167.179

In [4]:
# read the participant ids
pp_nrs = pd.read_csv('../../data/experimental_data/experiment_ppids.csv')['id']

Get the number of participants best fit by each model, as well as the strength of evidence for the best model over the second-best model for each participant

In [5]:
models_by_best_fitting_pps = defaultdict(int)
participant_to_best_model = {}
nm1_diff_by_pp = {}

for pp_id in pp_nrs:
    participant_fits = {}
    for agent_type in agent_types:
        # select the dataframe for the selected agent type
        df = all_model_dfs[agent_type]

        if len(df[df['pp_id'] == pp_id]['AIC']) != 1:
            print("len:", len(df[df['pp_id'] == pp_id]['AIC']))
            break
        
        # add the AIC to the participant fits dictionary
        participant_fits[agent_type] = float(df[df['pp_id'] == pp_id]['AIC'])
    
    if len(df[df['pp_id'] == pp_id]) != 1:
        continue
    
    # select the best-fitting agent for this participant
    sorted_fits = sorted(participant_fits.values())
    best_agent = min(participant_fits, key=participant_fits.get)
    nm1_diff =  np.mean([participant_fits[at] for at in agent_types if at != "null_model_1"]) - participant_fits["null_model_1"]
    
    # increment the number of pps best fit by the model
    models_by_best_fitting_pps[best_agent] += 1
    # store the best-fitting model for this participant
    participant_to_best_model[pp_id] = best_agent
    # save the difference between nm1 and the second best-fitting model
    nm1_diff_by_pp[pp_id] = nm1_diff

print(models_by_best_fitting_pps)

defaultdict(<class 'int'>, {'null_model_2': 16, 'hill_climbing': 37, 'sparse_max_discrete': 32, 'sparse_max_continuous': 14, 'sparse_lqr': 12})


In [6]:
# save the top 10 participants by nm1's fit
best_pps_nm1 = sorted(nm1_diff_by_pp, key=nm1_diff_by_pp.get, reverse=True)[:10]
df_nm1 = all_model_dfs["null_model_1"]
df_nm1 = df_nm1[df_nm1["pp_id"].isin(best_pps_nm1)]
df_nm1.to_csv("../../data/fitting_results/nm1_best_pps.csv", index=False)

Create csv files for Bayesian model selection (done using SPM8)

In [7]:
df_aics = pd.DataFrame()
for df_type in all_model_dfs:
    # convert AICs to log model evidence format
    df_aics[df_type] = all_model_dfs[df_type]['AIC'].apply(lambda x: -x/2)
df_aics.to_csv(f"{DATA_DIR}/aic_lme.csv")  # save to csv

n_params = {"null_model_2": 2, "null_model_1": 4, "lqr": 2, "sparse_lqr": 3, "hill_climbing": 3, "sparse_max_continuous": 4, "sparse_max_discrete": 4}

df_bics = pd.DataFrame()
for df_type in all_model_dfs:
    # convert BICs to log model evidence format
    df_bics[df_type] = all_model_dfs[df_type]["ll"].apply(lambda x: - (n_params[df_type] * np.log(10) - 2 * x) / 2)
df_bics.to_csv(f"{DATA_DIR}/bic_lme.csv")  # save to csv

In [8]:
# make a csv file with the best-fitting model and parameters for each participant
df_bestfit = pd.DataFrame()
for pp_id in pp_nrs:
    best_model_type = participant_to_best_model[pp_id]  # pick the best model type
    df = all_model_dfs[best_model_type]  # get the dataframe for the best model
    row = df[df['pp_id'] == pp_id]  # get the relevant row using the pp id
    df_bestfit = df_bestfit.append(row, ignore_index=True)  # add the selected row to the best fit dataframe
df_bestfit.to_csv(f"{DATA_DIR}/best_fitting_models.csv")

Get the mean best-fitting parameter for each model type

In [9]:
for model_type in df_bestfit['agent_type'].drop_duplicates():
    # get only the rows of the best-fit dataframe for the current model type
    df_model = df_bestfit[df_bestfit['agent_type'] == model_type]
    print(f"MODEL TYPE: {model_type}")
    print(f"exp param: {df_model['exp_param'].mean()}")
    print(f"vm param: {df_model['vm_param'].mean()}")
    if model_type in ("hill_climbing", "sparse_max_continuous", "sparse_max_discrete"):
        print(f"step size: {df_model['step_size'].mean()}")
    if model_type in ("sparse_lqr", "sparse_max_continuous", "sparse_max_discrete"):
        print(f"attention cost: {df_model['attention_cost'].mean()}")
    if model_type == "null_model_1":  # n and b parameters for nm1
        print(f"n: {np.round(df_model['n']).mean()}")
        print(f"b: {df_model['b'].mean()}")

MODEL TYPE: null_model_2
exp param: 0.0266682604176051
vm param: 6.083610428849356
MODEL TYPE: hill_climbing
exp param: 0.07337562077381873
vm param: 6.0093102211512015
step size: 0.33126701214033644
MODEL TYPE: sparse_max_discrete
exp param: 0.10871181966031433
vm param: 4.5227043630257295
step size: 0.6898139999425017
attention cost: 14.235119906069837
MODEL TYPE: sparse_max_continuous
exp param: 0.03556975179584271
vm param: 4.973135682639295
step size: 0.2719986741210881
attention cost: 12.254898554996227
MODEL TYPE: sparse_lqr
exp param: 0.05874843995317245
vm param: 4.40096609684599
attention cost: 150.69867445912084


## Compare Participant Scores by Best-Fitting Model

In [10]:
# read the participant data
raw_pp_data_path = '../../data/experimental_data/experiment_actions.csv'
df_pps = pd.read_csv(raw_pp_data_path)
df_last = df_pps.loc[df_pps.groupby("pp_id")['Unnamed: 0'].idxmax()]

Get scores by which model explains each pp best

In [11]:
scores_by_best_model = defaultdict(list)
for index, row in df_last.iterrows():
    scores_by_best_model[participant_to_best_model[row['pp_id']]].append(np.sqrt(row['total_cost']))

In [12]:
print(f"sparse lqr median: {np.median(scores_by_best_model['sparse_lqr'])}")
print(f"hill climbing median: {np.median(scores_by_best_model['hill_climbing'])}")

sparse lqr median: 95.97784526310616
hill climbing median: 114.59424069297725


In [13]:
def bootstrap_ci(data, n=1000000):
    all_medians = []
    for i in range(n):
        bs_data = np.random.choice(data, len(data), replace=True)
        med = np.median(bs_data)
        all_medians.append(med)
    all_medians = np.array(all_medians)
    lower_bound = np.percentile(all_medians, 2.5)
    upper_bound = np.percentile(all_medians, 97.5)
    
    return lower_bound, upper_bound

In [14]:
for model_type in ("sparse_lqr", "hill_climbing"):
    scores = scores_by_best_model[model_type]
    lower_bound, upper_bound = bootstrap_ci(scores)
    print(f"{model_type}: {np.median(scores)}, [{lower_bound}, {upper_bound}]")

sparse_lqr: 95.97784526310616, [30.046612351523706, 150.7278131887344]
hill_climbing: 114.59424069297725, [96.429404229208, 129.1154522123514]


In [15]:
stats.kruskal(scores_by_best_model['sparse_lqr'], scores_by_best_model['hill_climbing'])

KruskalResult(statistic=0.6248648648648611, pvalue=0.42924519572005637)

Get descriptive stats like number of variables manipulated.

In [16]:
print(len(scores_by_best_model['sparse_lqr']) + len(scores_by_best_model['hill_climbing']))

49


Compute the number of variables manipulated and input norm standard deviation for humans.